### Exercise #1

In [3]:
#importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
#Getting data from the pokemon page 
pokemon_url = 'https://pokemondb.net/'
# Issue a simple HTTP request to get the webpage text
pokemon_page = requests.get(pokemon_url)
# Response code is returned
pokemon_page

<Response [200]>

In [5]:
#parsing with BeautifulSoup library
pokemon_page_soup = BeautifulSoup(pokemon_page.text, 'html.parser')

In [6]:
#Getting data from the pokemondb pokedex page 
pokemondb_url = 'https://pokemondb.net/pokedex/all'
# Issue a simple HTTP request to get the webpage text
pokemondb_page = requests.get(pokemondb_url)
# Response code is returned
pokemondb_page

<Response [200]>

In [7]:
#parsing with BeautifulSoup library
pokemondb_page_soup = BeautifulSoup(pokemondb_page.text, 'html.parser')

In [8]:
#Finding all the tables with class data-table block-wide and Counting the total tables
pokemondb_table = pokemondb_page_soup.find_all(name="table", class_="data-table block-wide")
print('The number of tables in the list: ' + str(len(pokemondb_table)))

The number of tables in the list: 1


In [9]:
#pokemondb_page_soup

#### Step 1.1

In [10]:
# .find_all returns all matches in a list
pokemondb_values_list = pokemondb_page_soup.find_all(name = "tr" )
#print(pokemondb_values_list)
print("Number of rows including header: " + str(len(pokemondb_values_list)))

Number of rows including header: 927


In [11]:
bulbasaur = pokemondb_values_list[1]
#bulbasaur.prettify()

#### Step 1.2

In [12]:
#1.2)a)We could have used .find for the name of pokemon
print(bulbasaur.find(name="a", class_="ent-name").text)

Bulbasaur


In [13]:
#1.2)b)We could have used .find for the url"href"
url = bulbasaur.find(href= True)
link = url['href']
print("url to pokemon's page: https://pokemondb.net"+str(link))

url to pokemon's page: https://pokemondb.net/pokedex/bulbasaur


In [14]:
#1.2)c)We could have used .find for the class "cell-icon" to get type
type12 = bulbasaur.find(name="td", class_="cell-icon").text
type1 = type12.split()

In [15]:
type1

['Grass', 'Poison']

In [16]:
#1.2)d)We could have used .find for the class "cell-total" to get Total points
print(bulbasaur.find(name="td", class_="cell-total").text)

318


In [17]:
#1.2)e)

pokemon_data1 = [] #empty list to fill values in it.
pokemon = bulbasaur.find_all(name = 'td', class_ = 'cell-num') # cell-num is common to hp,attack,defense, etc. so will be enough to fetch all data and append in one list

length=len(pokemon) #using length of fetched elements in list so as to run the loop so as to append in list

for i in range(length):
    pokemon_data1.append(pokemon[i].text) #appending elements in list
    
print(pokemon_data1)   #printin 

['001', '45', '49', '49', '65', '65', '45']


#### Step 1.3

In [18]:
# defining fucntion which generalizes step 1.2 to give the results which will return a dataframe of single row

def pokemon_info(row=1): #default value for row is 1
    
    if(row>0):                                    #using this to skip the header row
        pokemon_ = pokemondb_values_list[row]     #taking information about mentioned row's pokemon
        pokemon_data = []                         #initializing empty list 
        
        pokemon_data.append(pokemon_.find(name="a", class_="ent-name").text) #appending name of pokemon to list elements 
        
        url_data = pokemon_.find(href= True)
        link_data = "https://pokemondb.net"+str(url_data['href'])
        pokemon_data.append(link_data)                                       #appending url of pokemon to list elements
        
        typemix=pokemon_.find(name="td", class_="cell-icon").text
        typesingle=typemix.split()
        pokemon_data.append(typesingle)                                      #appending type of pokemon to list     
        
        pokemon_data.append(pokemon_.find(name="td", class_="cell-total").text) #appedning cell total to list
        
        pokemon_data1 = []
        pokemon = pokemon_.find_all(name = 'td', class_ = 'cell-num')        
        length=len(pokemon)
        for i in range(length):
            pokemon_data1.append(pokemon[i].text)
            
        pokemon_data.extend(pokemon_data1)                                  #extending results from 1.2)e) to list
        
        return pd.DataFrame(pokemon_data).T                                 #converting list into dataframe and returning as single row
    
    else:
        print("You are giving a wrong row number")                          

In [19]:
tot_rows=len(pokemondb_values_list)
tot_rows

927

### Exercise #2

In [20]:
pok_df =pokemon_info(1)  # taking information for first row

In [21]:
pok_df      # dataframe for first row

,0,1,2,3,4,5,6,7,8,9,10
0,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,001,45,49,49,65,65,45


In [22]:
pok_df = pd.DataFrame()

for i in range(tot_rows):      #Extracting every row from table and adding it to dataframe
    i=i+1
    if i<927:
        pok_df1=pokemon_info(i)
        
    pok_df=pok_df.append(pok_df1,ignore_index=True)    
    

In [23]:
pok_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,001,45,49,49,65,65,45
1,Ivysaur,https://pokemondb.net/pokedex/ivysaur,"[Grass, Poison]",405,002,60,62,63,80,80,60
2,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",525,003,80,82,83,100,100,80
3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",625,003,80,100,123,122,120,80
4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,004,39,52,43,60,50,65


#### Step 2.1

In [24]:
#Add column names to dataframe
pok_df.columns = ['Name', 'URL', 'Type/s','Total points','ID Number','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']

In [25]:
pok_df[pok_df['Name']=="Bulbasaur"].URL

0    https://pokemondb.net/pokedex/bulbasaur
Name: URL, dtype: object

In [26]:
#converting strings to numeric where needed
pok_df['Total points']=pok_df['Total points'].astype(int)
pok_df['ID Number']=pok_df['ID Number'].astype(int)
pok_df['HP']=pok_df['HP'].astype(int)
pok_df['Attack']=pok_df['Attack'].astype(int)
pok_df['Defense']=pok_df['Defense'].astype(int)
pok_df['Sp. Atk']=pok_df['Sp. Atk'].astype(int)
pok_df['Sp. Def']=pok_df['Sp. Def'].astype(int)
pok_df['Speed']=pok_df['Speed'].astype(int)

In [27]:
# Making ID number first column in dataframe 

In [28]:
cols = list(pok_df.columns.values)

In [29]:
cols

['Name',
 'URL',
 'Type/s',
 'Total points',
 'ID Number',
 'HP',
 'Attack',
 'Defense',
 'Sp. Atk',
 'Sp. Def',
 'Speed']

In [30]:
columnsTitles = ['ID Number','Name','URL','Type/s','Total points','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']

pok_df = pok_df.reindex(columns=columnsTitles)

In [31]:
pok_df.head()

,ID Number,Name,URL,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,45,49,49,65,65,45
1,2,Ivysaur,https://pokemondb.net/pokedex/ivysaur,"[Grass, Poison]",405,60,62,63,80,80,60
2,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",525,80,82,83,100,100,80
3,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",625,80,100,123,122,120,80
4,4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,39,52,43,60,50,65


#### Step 2.2 

In [32]:
pok_df_dummy=pok_df['Type/s'].str.join(sep=',').str.get_dummies(sep=',') #using column 'Type/s' to make dummy variables by separating 2 values in one column by comma and uniquely creating dummy variable for it. 

In [33]:
pok_df_result = pd.concat([pok_df,pok_df_dummy], axis=1) #adding dummy variable columns with result to right of dataframe

In [34]:
pok_df_result.head()

,ID Number,Name,URL,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,45,49,49,65,65,...,0,1,0,0,0,1,0,0,0,0
1,2,Ivysaur,https://pokemondb.net/pokedex/ivysaur,"[Grass, Poison]",405,60,62,63,80,80,...,0,1,0,0,0,1,0,0,0,0
2,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",525,80,82,83,100,100,...,0,1,0,0,0,1,0,0,0,0
3,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",625,80,100,123,122,120,...,0,1,0,0,0,1,0,0,0,0
4,4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,39,52,43,60,50,...,0,0,0,0,0,0,0,0,0,0


#### Step 2.3

In [35]:
without_duplicates=pok_df.drop_duplicates('URL') #removing duplicates by using url and keeping first occuring value
print("Number of rows:" + str(len(without_duplicates)))

Number of rows:809


In [36]:
without_duplicates['Name']=="Charizard"

0      False
1      False
2      False
4      False
5      False
6       True
9      False
10     False
11     False
13     False
14     False
15     False
16     False
17     False
18     False
20     False
21     False
22     False
24     False
26     False
28     False
29     False
30     False
31     False
32     False
34     False
36     False
38     False
40     False
41     False
       ...  
893    False
894    False
895    False
896    False
897    False
898    False
899    False
900    False
901    False
902    False
903    False
904    False
905    False
906    False
907    False
908    False
909    False
910    False
911    False
912    False
913    False
917    False
918    False
919    False
920    False
921    False
922    False
923    False
924    False
925    False
Name: Name, Length: 809, dtype: bool

#### Step 2.4

In [37]:
pok_df["sample"] =((pok_df["ID Number"]%4)==0).astype(int) # adding column sample which acts as dummy variable giving 1 for ppokemon which has ID number as multiple of 4

In [38]:
without_duplicates["sample"] =((pok_df["ID Number"]%4)==0).astype(int) # adding column sample which acts as dummy variable giving 1 for ppokemon which has ID number as multiple of 4

C:\Users\sushi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
pok_df.head() #displaying with duplicates dataframe with sample dummy variable

,ID Number,Name,URL,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sample
0,1,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,45,49,49,65,65,45,0
1,2,Ivysaur,https://pokemondb.net/pokedex/ivysaur,"[Grass, Poison]",405,60,62,63,80,80,60,0
2,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",525,80,82,83,100,100,80,0
3,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",625,80,100,123,122,120,80,0
4,4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,39,52,43,60,50,65,1


In [40]:
without_duplicates.head() #displaying without duplicates dataframe with sample dummy variable

,ID Number,Name,URL,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sample
0,1,Bulbasaur,https://pokemondb.net/pokedex/bulbasaur,"[Grass, Poison]",318,45,49,49,65,65,45,0
1,2,Ivysaur,https://pokemondb.net/pokedex/ivysaur,"[Grass, Poison]",405,60,62,63,80,80,60,0
2,3,Venusaur,https://pokemondb.net/pokedex/venusaur,"[Grass, Poison]",525,80,82,83,100,100,80,0
4,4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,39,52,43,60,50,65,1
5,5,Charmeleon,https://pokemondb.net/pokedex/charmeleon,[Fire],405,58,64,58,80,65,80,0


In [41]:
sample_df = without_duplicates[without_duplicates['sample']==1] #selecting only the rows with sample column value as 1 and adding it to new dataframe sample_df

In [42]:
sample_df.head()

,ID Number,Name,URL,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sample
4,4,Charmander,https://pokemondb.net/pokedex/charmander,[Fire],309,39,52,43,60,50,65,1
10,8,Wartortle,https://pokemondb.net/pokedex/wartortle,[Water],405,59,63,80,65,80,58,1
15,12,Butterfree,https://pokemondb.net/pokedex/butterfree,"[Bug, Flying]",395,60,45,50,90,80,70,1
20,16,Pidgey,https://pokemondb.net/pokedex/pidgey,"[Normal, Flying]",251,40,45,40,35,35,56,1
26,20,Raticate,https://pokemondb.net/pokedex/raticate,[Normal],413,55,81,60,50,70,97,1


### Exercise #3

#### Step 3.1

In [52]:
pokemondb_values_list

[<tr>
 <th class="sorting" data-sort="int"><div class="sortwrap">#</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Name</div></th> <th><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Total</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">HP</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Attack</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Defense</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Atk</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Sp. Def</div></th> <th class="sorting" data-sort="int"><div class="sortwrap">Speed</div></th> </tr>,
 <tr>
 <td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><span class="img-fixed icon-pkmn" data-alt="Bulbasaur icon" data-src="https://img.pokemondb.net/sprites/sun-moon/icon/bulbasaur.png"></span></span><span class="infocard-cell-da

In [50]:
row1

<tr>
<td class="cell-num cell-fixed" data-sort-value="1"><span class="infocard-cell-img"><span class="img-fixed icon-pkmn" data-alt="Bulbasaur icon" data-src="https://img.pokemondb.net/sprites/sun-moon/icon/bulbasaur.png"></span></span><span class="infocard-cell-data">001</span></td> <td class="cell-name"><a class="ent-name" href="/pokedex/bulbasaur" title="View pokedex for #001 Bulbasaur">Bulbasaur</a></td><td class="cell-icon"><a class="type-icon type-grass" href="/type/grass">Grass</a><br/> <a class="type-icon type-poison" href="/type/poison">Poison</a></td>
<td class="cell-total">318</td>
<td class="cell-num">45</td>
<td class="cell-num">49</td>
<td class="cell-num">49</td>
<td class="cell-num">65</td>
<td class="cell-num">65</td>
<td class="cell-num">45</td>
</tr>

In [51]:
link_data_row1

'https://pokemondb.net/pokedex/bulbasaur'

In [43]:
row1=pokemondb_values_list[1]   #here, i am trying to get image by first going into table values of dataframe with duplicates using row number    
url_data_row1= row1.find(href= True) #here, i am getting url for the mentioned row number pokemon
link_data_row1 = "https://pokemondb.net"+str(url_data_row1['href']) #here, i am attaching particular pokemon url to general form
pokemon_data1=link_data_row1
image_name=str.lower(row1.find(name="a", class_="ent-name").text) #lower casing the name
name = image_name.capitalize()

In [44]:
pokemon_data1

'https://pokemondb.net/pokedex/bulbasaur'

In [45]:
#Getting data from the pokemon page 
imageurl = pokemon_data1
# Issue a simple HTTP request to get the webpage text
imageurl_page = requests.get(imageurl)
# Response code is returned
imageurl_page

<Response [200]>

In [46]:
#parsing with BeautifulSoup library
imageurl_page_soup = BeautifulSoup(imageurl_page.text, 'html.parser')

In [47]:
def image1(name1):
    name2="https://img.pokemondb.net/artwork/"+name1+".jpg"
    pokemon_image1 = imageurl_page_soup.find_all(name="img",src=name2)
    length1=len(pokemon_image1)
    loop = []
    for i in range(length1):
        final_image=pokemon_image1[i]
        final_image1=final_image["src"]
        loop.append(final_image1)
    return loop

In [48]:
from IPython.display import Image
from IPython.core.display import HTML

In [49]:
#source = "https://img.pokemondb.net/artwork/large/"+image_name+".jpg"
ans1 = image1(image_name)
lengthtry=len(ans1)
for i in range(lengthtry):
    print(ans1[i])
    display(Image(url=ans1[i], unconfined=True))

https://img.pokemondb.net/artwork/bulbasaur.jpg


#### Step 3.2

In [53]:
pokemon_data1

'https://pokemondb.net/pokedex/bulbasaur'

In [701]:
tables = pd.read_html(requests.get(pokemon_data1, headers={'User-agent': 'Mozilla/5.0'}).text)

In [702]:
table_desired=pd.DataFrame(tables[15]) #using index 15 to extract the required table

In [695]:
table_desired.head()

,0,1
0,RedBlue,Pallet Town
1,Yellow,Cerulean City
2,GoldSilverCrystal,Trade/migrate from another game
3,RubySapphire,Trade/migrate from another game
4,FireRedLeafGreen,Pallet Town


#### Step 3.3

In [703]:
transpose=table_desired.T #using T to tranpose dataframe
transpose

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,RedBlue,Yellow,GoldSilverCrystal,RubySapphire,FireRedLeafGreen,Emerald,DiamondPearlPlatinum,HeartGoldSoulSilver,BlackWhiteBlack 2White 2,XY,Omega RubyAlpha Sapphire,SunMoon,Ultra SunUltra Moon,Let's Go PikachuLet's Go Eevee
1,Pallet Town,Cerulean City,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Trade/migrate from another game,Pallet Town,Trade/migrate from another game,Lumiose City,Trade/migrate from another game,Trade/migrate from another game,Route 2,"Cerulean City, Viridian Forest"


#### Step 3.4 

In [704]:
import time
List = []
List = list(sample_df["URL"])
holdurl = []
holdloc = []
print(len(List))
for i in List:
    tables2 = pd.read_html(requests.get(i, headers={'User-agent': 'Mozilla/5.0'}).text)
    time.sleep(0.001)
    transpose1 = tables2[-2].T
    frame=pd.DataFrame(transpose1)
    header = frame.iloc[0]
    
    frame = frame[1:] #rest of data minus header
    frame.columns = header #header row as df header
    
    try:                                #using try and except for Meltan
        select = list(frame["XY"])
        holdurl.append(i)
        holdloc.append(select)
    except:
        continue

202


In [678]:
H = pd.DataFrame(holdurl)
H = H.rename(columns = {0:"URL"})    #Making URL as column header
H2 =pd.DataFrame(holdloc)
H2 = H2.rename(columns = {0:"XY"})   #Making XY as column header
result = pd.concat([H,H2], axis =1, join ="inner") #concatinating using inner join
result

,URL,XY
0,https://pokemondb.net/pokedex/charmander,Lumiose City
1,https://pokemondb.net/pokedex/wartortle,Evolve Squirtle
2,https://pokemondb.net/pokedex/butterfree,Evolve Caterpie/Metapod
3,https://pokemondb.net/pokedex/pidgey,"Route 2, 3"
4,https://pokemondb.net/pokedex/raticate,Trade/migrate from another game
5,https://pokemondb.net/pokedex/arbok,Route 19
6,https://pokemondb.net/pokedex/sandslash,"Route 18, Terminus Cave"
7,https://pokemondb.net/pokedex/nidoran-m,Route 11
8,https://pokemondb.net/pokedex/clefable,Trade/migrate from another game
9,https://pokemondb.net/pokedex/gloom,Evolve Oddish


### Exercise #4

#### Step 4.1

In [465]:
def analysis1(Types):
    avg_attack = []
    avg_defense = []
    types1 = []
    op = []
    for t in Types:
        avg_attack.append(pok_df_result[pok_df_result[t]==1].Attack.mean()) #appedning to avg_attack list
        avg_defense.append(pok_df_result[pok_df_result[t]==1].Defense.mean()) #appedning to avg_defense list
        types1.append(t) #appedning to types1 list
        
    op=pd.concat([pd.DataFrame(types1),pd.DataFrame(avg_attack),pd.DataFrame(avg_defense)],axis=1,ignore_index=True)
    op1 = pd.DataFrame(op)
    op2 = op1
    return op2

In [699]:
Types=pok_df_dummy.columns.values
analysisone=analysis1(Types) #passing type to function analysis1 
analysisone.columns=["Type","Avg. Attack", "Avg. Defense"] #making column names as Type,Avg. Attack and Avg. Defense
analysisone.head()

,Type,Avg. Attack,Avg. Defense
0,Bug,72.734940,72.168675
1,Dark,95.066667,69.216667
2,Dragon,103.475410,87.393443
3,Electric,73.741935,69.693548
4,Fairy,64.641509,73.113208


In [467]:
analysisone.loc[analysisone['Avg. Attack'].idxmax()]

Type            Fighting
Avg. Attack      105.825
Avg. Defense     75.3968
Name: 5, dtype: object

In [468]:
analysisone.loc[analysisone['Avg. Attack'].idxmin()]

Type              Fairy
Avg. Attack     64.6415
Avg. Defense    73.1132
Name: 4, dtype: object

In [469]:
analysisone.loc[analysisone['Avg. Defense'].idxmax()]

Type              Steel
Avg. Attack     97.1587
Avg. Defense    114.635
Name: 16, dtype: object

In [470]:
analysisone.loc[analysisone['Avg. Defense'].idxmin()]

Type             Normal
Avg. Attack     74.5556
Avg. Defense    60.2564
Name: 12, dtype: object

#### 1)Highest avg. attack is for Fighting type as 105.825.
#### 2)Lowest avg. attack is for Fairy type as 64.6415
#### 3)Highest avg. defense is for Steel type as 114.635
#### 4)Lowest avg. defense is for Normal type as 60.2564

#### Step 4.2

In [499]:
analysis2 = pd.merge(result,sample_df, on='URL',how="inner")

In [504]:
analysis2.tail()

,URL,XY,ID Number,Name,Type/s,Total points,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sample
136,https://pokemondb.net/pokedex/doublade,Evolve Honedge,680,Doublade,"[Steel, Ghost]",448,59,110,150,45,49,35,1
137,https://pokemondb.net/pokedex/tyrunt,Ambrette Town,696,Tyrunt,"[Rock, Dragon]",362,58,89,77,45,45,48,1
138,https://pokemondb.net/pokedex/goomy,Route 14,704,Goomy,[Dragon],300,45,50,35,55,75,40,1
139,https://pokemondb.net/pokedex/phantump,Route 16,708,Phantump,"[Ghost, Grass]",309,43,70,48,50,60,38,1
140,https://pokemondb.net/pokedex/bergmite,Frost Cavern,712,Bergmite,[Ice],304,55,69,85,32,35,28,1


In [545]:
analysistwo=analysis2.groupby('XY').mean()
#analysistwo['Total points']
analysistwo.loc[analysistwo['Total points'].idxmax()]


ID Number       144.0
Total points    580.0
HP               90.0
Attack           85.0
Defense         100.0
Sp. Atk          95.0
Sp. Def         125.0
Speed            85.0
sample            1.0
Name: Sea Spirit's Den, Roaming Kalos, dtype: float64

#### Location is Sea Spirit's Den, Roaming Kalos with highest average total point score as 580

### Extra work : If you want to get images of Charizard mega x, y and such duplicates existing

In [ ]:
def image1(name1):
    name2="https://img.pokemondb.net/artwork/"+name1+".jpg"
    pokemon_image1 = imageurl_page_soup.find_all(name="a",rel="lightbox",href=True)
    length1=len(pokemon_image1)
    loop = []
    for i in range(length1):
        final_image=pokemon_image1[i]
        final_image1=final_image["href"]
        loop.append(final_image1)
    return loop